CHARGEMENT MODEL RESNET FB from HUGGING FACE

ET DETECTION OBJET SUR VIDEO

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import cv2
import numpy as np

# CHARGEMENT MODELE
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

# DETECTION DES OBJETS DANS LE FLUX VIDEO
def detect_objects(frame):
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    # Récupérer les classes et scores
    logits = outputs.logits[0]
    boxes = outputs.pred_boxes[0]

    threshold = 0.9
    labels = logits.softmax(-1).argmax(-1)
    scores = logits.softmax(-1).max(-1).values

    detected = []
    for box, label, score in zip(boxes, labels, scores):
        if score > threshold:
            detected.append((box.tolist(), label.item(), score.item()))

    return detected


AFFICHAGE VIDEO

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    detected = detect_objects(frame)
    for box, label, score in detected:
        x, y, w, h = box
        cv2.rectangle(frame, (int(x), int(y)), (int(w), int(h)), (255, 0, 0), 2)
        cv2.putText(frame, f"Label: {label} ({score:.2f})", 
                    (int(x), int(y) - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    cv2.imshow('Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
